# Задача
Итак, давайте сформулируем задачу: необходимо написать библиотеку для отправки сообщений с помощью в Телеграм, указывая в качестве адресата свой логин. Т. е. вместо импорта библиотек, токена бота, вспоминания своего chat_id и написания методов мы хотим прийти к следующим простым командам вида ('kbashevoy' - имя пользователя, которому отправляем сообщение 'Посчиталось ура'):
```python
mess = notifier('kbashevoy')

mess.notification('Посчиталось ура')
```
И получить в своем Телеграме это сообщение.

# Основы объектно-ориентированного программирования
Для начала откроем файл config.yaml с прошлого шага, в который добавим соответствие вашего chat_id (тоже с прошлого шага) и удобного вам логина. Т. к. пользователей нашей библиотеки может быть много, то давайте все пары chat_id и логинов запишем в ключ users. Итого, файл config.yaml будет иметь вид (для простоты обозначаем самого себя в списке пользователей как 'me'):

In [25]:
import yaml
import telegram
import sys

In [21]:
path_to_cfg = './module15_files/telegram_config_2.yaml'

with open(path_to_cfg) as f:
    config = yaml.load(f)

In [8]:
class Notifier:
    
    def __init__(self, username):
        self.username = username

In [9]:
mess = Notifier('me')

In [10]:
mess.username

'me'

Мы завели так называемый инстанс класса Notifier. Это очень удобно, т. к. если вам нужно завести "рассыльщики" сообщений для нескольких пользователей, то вы можете сделать это аналогичным образом:Мы завели так называемый инстанс класса Notifier. Это очень удобно, т. к. если вам нужно завести "рассыльщики" сообщений для нескольких пользователей, то вы можете сделать это аналогичным образом:

In [11]:
second_mess = Notifier('my colleague')

In [12]:
print(second_mess.username)

my colleague


Давайте в качестве иллюстрации этого свойства сделаем счетчик отправленных сообщений. Сначала объявим переменную messages_counter, которая будет считать количество отправленных сообщений (класс придется все время писать целиком и в одной ячейке):

In [13]:
class Notifier:
    
    def __init__(self, username):
        self.username = username
        self.messages_counter = 0
        
    def counterUp(self):
        self.messages_counter += 1

In [14]:
mess = Notifier('me')
print(mess.messages_counter)
mess.counterUp()
print(mess.messages_counter)

0
1


Это гораздо проще, чем заводить очередную переменную в цикле. Если в счетчике у вас заложена более сложна логика работы (например, вместо увеличения на 1 у вас идет начисление премий по отделам), то в итоге в коде вместо описания правил вы будете просто вызывать этот метод.

Наконец, добавляем необходимые переменные и команды для работы с ботом. Обратите внимание, что метод notification (он отправляет сообщение в Телеграм) содержит дополнительный аргумент message (текст сообщения). Этот аргумент будет доступен пока только методу notification. Также будем проверять наличие пользователя username в словаре config['users']

In [23]:
class Notifier:

    def __init__(self, username, path):
        self.username = username        
        self.messages_counter = 0
        
        with open(path) as f:
            self.config = yaml.load(open(path, 'r'))
        
        self.bot = telegram.Bot(self.config['bot_token']) 

    def counterUp(self):
        self.messages_counter += 1
    
    def notification(self, message):
        if self.username in self.config['users']:
            self.bot.sendMessage(chat_id = self.config['users'][self.username], text = message)
        else:
            print('Имя пользователя не найдено в файле config.yaml')

In [24]:
mess = Notifier('unknown_user', path_to_cfg)
mess.notification('test')

Имя пользователя не найдено в файле config.yaml


# Использование в других скриптах
Чтобы ваша библиотека была доступна из других скриптов, давайте сохраним ее отдельный файл. Для "чистоты эксперимента" сохраните ее в отдельную папку 'notifier' в файл telegram_notifier.py. В эту же папку положите файл с токеном и логинами config.yaml. Тем самым мы проверим как вызывать нашу библиотеку из другой папки. Итоговый вид библиотеки в файле telegram_notifier.py:

In [26]:
!cd

F:\My_projects\Programming_courses\jupyter\Skill_factory\module15


In [ ]:
# указываем скрипту где искать нашу библиотеку

import sys

sys.path.append('notifier')

# импортируем класс Notifier из файла telegram_notifier (без указания расширения py)

from telegram_notifier import Notifier

Все готово к использованию! Отправим себе служебное сообщение в 2 строчки кода:

mess = Notifier('me')

mess.notification('Расчет окончен')

Теперь вы можете аналогичным образом вызывать эту библиотеку из любого скрипта. Не забывайте прописывать "sys.path.append('notifier')" с указанием полного адреса папки 'notifier'.